###### setting work path

In [1]:
import os

print(os.getcwd())
os.chdir(r'C:\Users\dongwan.kim\Desktop\da_work\2000_trueskill')
print(os.getcwd())

D:\_PlayGround\Github\pg\_chaos\matching
C:\Users\dongwan.kim\Desktop\da_work\2000_trueskill


###### loading file

In [4]:
import pandas as pd

file_name = '20180711_ratingSim.xlsx'
df = pd.read_excel(io=file_name, sheet_name='match_result', header=0)
df

,reg_date,matchid,teamno,playerid,result,weight
0,2018-07-01 11:00:00,1,0,7,0,1
1,2018-07-01 11:00:00,1,0,6,0,1
2,2018-07-01 11:00:00,1,1,3,1,1
3,2018-07-01 11:00:00,1,1,5,1,1
4,2018-07-01 11:00:00,2,0,6,0,1
5,2018-07-01 11:00:00,2,1,3,1,1


In [5]:
header_names = list(df)
header_names

['reg_date', 'matchid', 'teamno', 'playerid', 'result', 'weight']

###### trueskill settings

In [6]:
import trueskill as ts
import matplotlib.pyplot as plt
import numpy as np
import math
from scipy.stats import norm
pdf = norm.pdf
cdf = norm.cdf
icdf = norm.ppf  # inverse CDF

def get_draw_margin(p, beta, total_players=2):
    """ Compute the draw margin (epsilon) given the draw probability. """
    return icdf((p + 1.0) / 2) * math.sqrt(total_players) * beta

init_mu = 25
init_sigma = init_mu / 3
init_beta = init_sigma / 2
init_gamma = init_sigma / 100
init_draw_probability = 0.1
init_epsilon = get_draw_margin(0.1, init_beta); init_epsilon

env = ts.TrueSkill(mu=init_mu, sigma=init_sigma, beta=init_beta, tau=init_gamma, draw_probability=init_draw_probability, backend='scipy')

###### df transformer

In [41]:
import pandasql as pdsql


def get_transformed_df(df, col_name_regdate, col_name_matchid):
    query = """
    select
        t2.match_min_reg_date
        , t1.*
    from 
        df as t1
        inner join
        (
            select 
                """ + col_name_matchid + """
                , min(""" + col_name_regdate + """) as match_min_reg_date
            from 
                df
            group by 
                """ + col_name_matchid + """ 
        ) as t2
        on
            t1.""" + col_name_matchid + """ = t2.""" + col_name_matchid + """
    order by
        t2.match_min_reg_date
        , t1.""" + col_name_matchid + """
    """
    return pdsql.sqldf(query, locals())   

In [42]:
# test code

import pandas as pd


file_name = '20180711_ratingSim.xlsx'
dfx = pd.read_excel(io=file_name, sheet_name='match_result', header=0)
get_transformed_df(dfx, 'reg_date', 'matchid')

,match_min_reg_date,reg_date,matchid,teamno,playerid,result,weight
0,2018-07-01 11:00:00.000000,2018-07-01 11:00:00.000000,1,0,7,0,1
1,2018-07-01 11:00:00.000000,2018-07-01 11:00:00.000000,1,0,6,0,1
2,2018-07-01 11:00:00.000000,2018-07-01 11:00:00.000000,1,1,3,1,1
3,2018-07-01 11:00:00.000000,2018-07-01 11:00:00.000000,1,1,5,1,1
4,2018-07-01 11:00:00.000000,2018-07-01 11:00:00.000000,2,0,6,0,1
5,2018-07-01 11:00:00.000000,2018-07-01 11:00:00.000000,2,1,3,1,1


###### slicing

In [43]:
def get_itermatch(df, col_name_regdate, col_name_matchid, col_name_teamno):
    """
    df: Pandas dataframe
    """
    def append_player(match, player):
        team_no = player[col_name_teamno]
        if match.get(team_no) is None:  # new team encountered
            match[team_no] = [player]
        else:
            match[team_no].append(player)
            
    last_matchid = None
    match = dict()
    
    df = get_transformed_df(df, col_name_regdate, col_name_matchid)
    
    for idx, row in df.iterrows():
        player = row.to_dict()
        this_matchid = player[col_name_matchid]

        if idx == 0 or last_matchid == this_matchid:  # 1st record or same match with last record
            append_player(match, player)
        else:  # new match record started
            yield match
            match = dict()
            append_player(match, player)
        
        last_matchid = this_matchid
        
    yield match

In [36]:
# test code

file_name = '20180711_ratingSim.xlsx'
dfx = pd.read_excel(io=file_name, sheet_name='match_result', header=0)

itermatch = get_itermatch(dfx, 'reg_date', 'matchid', 'teamno')
list(itermatch)

[{0: [{'match_min_reg_date': 1,
    'matchid': 1,
    'playerid': 7,
    'reg_date': '2018-07-01 11:00:00.000000',
    'result': 0,
    'teamno': 0,
    'weight': 1},
   {'match_min_reg_date': 1,
    'matchid': 1,
    'playerid': 6,
    'reg_date': '2018-07-01 11:00:00.000000',
    'result': 0,
    'teamno': 0,
    'weight': 1}],
  1: [{'match_min_reg_date': 1,
    'matchid': 1,
    'playerid': 3,
    'reg_date': '2018-07-01 11:00:00.000000',
    'result': 1,
    'teamno': 1,
    'weight': 1},
   {'match_min_reg_date': 1,
    'matchid': 1,
    'playerid': 5,
    'reg_date': '2018-07-01 11:00:00.000000',
    'result': 1,
    'teamno': 1,
    'weight': 1}]},
 {0: [{'match_min_reg_date': 2,
    'matchid': 2,
    'playerid': 6,
    'reg_date': '2018-07-01 11:00:00.000000',
    'result': 0,
    'teamno': 0,
    'weight': 1}],
  1: [{'match_min_reg_date': 2,
    'matchid': 2,
    'playerid': 3,
    'reg_date': '2018-07-01 11:00:00.000000',
    'result': 1,
    'teamno': 1,
    'weight': 1}]}

###### here we go

In [33]:
itermatch = get_itermatch(dfx, 'reg_date', 'matchid', 'teamno')

for match in itermatch:  # for each match(dict those key is teamno.)
    print(match)
    
    rating_groups = []
    
    for team_no, player in match.items():
        pass
        

{0: [{'match_min_reg_date': 1, 'reg_date': '2018-07-01 11:00:00.000000', 'matchid': 1, 'teamno': 0, 'playerid': 7, 'result': 0, 'weight': 1}, {'match_min_reg_date': 1, 'reg_date': '2018-07-01 11:00:00.000000', 'matchid': 1, 'teamno': 0, 'playerid': 6, 'result': 0, 'weight': 1}], 1: [{'match_min_reg_date': 1, 'reg_date': '2018-07-01 11:00:00.000000', 'matchid': 1, 'teamno': 1, 'playerid': 3, 'result': 1, 'weight': 1}, {'match_min_reg_date': 1, 'reg_date': '2018-07-01 11:00:00.000000', 'matchid': 1, 'teamno': 1, 'playerid': 5, 'result': 1, 'weight': 1}]}
{0: [{'match_min_reg_date': 2, 'reg_date': '2018-07-01 11:00:00.000000', 'matchid': 2, 'teamno': 0, 'playerid': 6, 'result': 0, 'weight': 1}], 1: [{'match_min_reg_date': 2, 'reg_date': '2018-07-01 11:00:00.000000', 'matchid': 2, 'teamno': 1, 'playerid': 3, 'result': 1, 'weight': 1}]}
